Topic Modelling

In [4]:
import spacy
import re
import spacy_transformers
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators
import torch

import json

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
nlp = spacy.load('de_dep_news_trf')

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

/home/somber/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'de_dep_news_trf' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to /home/somber/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
pipeline = spacy_cleaner.Pipeline(
    nlp,
    removers.remove_stopword_token,
    replacers.replace_punctuation_token,
    mutators.mutate_lemma_token,
)

In [6]:
with open('BundestagsReden.json') as json_file:
    data = json.load(json_file)

Lowercase is already done during forming dictionary

In [70]:
print(data['ID20100600'][1])

sehr geehrter herr alterspräsident dr. schäuble! also, herr brandner, gleich in der ersten sitzung die braune widerlichkeitsskala in solche höhen zu treiben,das ist immerhin respektvoll.dass ausgerechnet sie willy brandt zitieren, der gegen die nazis gekämpft hat, der niedergekniet ist vor den opfern des aufstandes im warschauer ghetto, dass ausgerechnet sie, die in der tradition der nazis stehen,das hier anbringen, ist abartig, um das in aller klarheit zu sagen. abartig und ekelerregend!liebe kolleginnen und kollegen, sehr geehrte damen und herren von der afd,wenn ich mir die politische lage angucke, dann finde ich zurzeit besonders wichtig: wie entwickelt sich corona? ich finde es wichtig, darüber zu diskutieren, wie wir kinderarmut bekämpfen können. und ich mache mir große sorgen, dass sich hier eine koalition anbahnt, in der das soziale hinten runterfallen wird, weil die stabil marktradikale fdp dafür schon sorgen wird.das ist etwas, was mich und viele menschen bewegt.was ist der a

In [15]:
doc = nlp(data['ID20100600'][1].replace('–', ''))

In [16]:
# doc = nlp(data)

Tokenize

In [29]:
for token in doc:
    if (not token.is_punct 
    and not token.is_currency
    and not token.is_digit
    and not token.is_space
    and not token.is_stop
    and not token.like_num):
        print(token.lemma_)

geehrt
Herr
Alterspräsident
dr
Schäuble
Herr
Brandner
Sitzung
braun
Widerlichkeitsskala
Höhe
treiben
immerhin
respektvoll.dass
ausgerechnet
Willy
Brandt
zitieren
Nazi
kämpfen
niederknieen
Opfer
Aufstand
Warschauer
Ghetto
ausgerechnet
Tradition
Nazi
stehen
anbringen
abartig
Klarheit
sagen
abartig
ekelerregend
lieb
Kollegin
Kollege
geehrt
Dame
Herren
afd
politisch
Lage
angucken
finden
zurzeit
wichtig
entwickeln
Corona
finden
wichtig
diskutieren
Kinderarmut
bekämpfen
machen
Sorge
Koalition
anbahnen
sozial
hinten
runterfallen
stabil
marktradikal
FDP
sorgen
wird.das
Mensch
bewegt.was
afd
wichtig
afd
wichtig
Verunglimpfung
geschlechtergerecht
Sprache
einzig
sinnvoll
Änderungsantrag
sehen
parlamentarisch
verblödet
eigentlich
mal
sagen
Änderungsantrag
einzig
erkennen
offenbar
Bundestag
Gesetzestext
lesen
binnen-i
doppelpunkte
gendersternchen.das
absurd
beantragen
Gesetzestext
völlig
Papierkorb
vorlegen
Fraktion
Vorlage
sozialdemokratisch
Fraktion
unterstützen
halten
Geschäftsordnung
Grundlage


In [30]:
for token in doc:
    print(token.lemma_)

sehr
geehrt
Herr
Alterspräsident
dr
.
Schäuble
--
also
--
Herr
Brandner
--
gleich
in
der
erster
Sitzung
der
braun
Widerlichkeitsskala
in
solcher
Höhe
zu
treiben
--
der
sein
immerhin
respektvoll.dass
ausgerechnet
sie
Willy
Brandt
zitieren
--
der
gegen
der
Nazi
kämpfen
haben
--
der
niederknieen
sein
vor
der
Opfer
der
Aufstand
in
Warschauer
Ghetto
--
dass
ausgerechnet
sie
--
der
in
der
Tradition
der
Nazi
stehen
--
der
hier
anbringen
--
sein
abartig
--
um
der
in
aller
Klarheit
zu
sagen
--
abartig
und
ekelerregend
--
lieb
Kollegin
und
Kollege
--
sehr
geehrt
Dame
und
Herren
von
der
afd
--
wenn
ich
sich
der
politisch
Lage
angucken
--
dann
finden
ich
zurzeit
besonders
wichtig
--
wie
entwickeln
sich
Corona
--
ich
finden
es
wichtig
--
darüber
zu
diskutieren
--
wie
wir
Kinderarmut
bekämpfen
können
--
und
ich
machen
mir
groß
Sorge
--
dass
sich
hier
ein
Koalition
anbahnen
--
in
der
der
sozial
hinten
runterfallen
werden
--
weil
der
stabil
marktradikal
FDP
dafür
schon
sorgen
wird.das
sein
etwas
--
we

def preprocessing (textToProcess):
    #tokenize
    #remove special characters
    #remove stopwords
    #lemmata
    processedText = pipeline.clean(textToProcess)
    return processedText

In [72]:
#iterate over all texts
alltexts = []
tokenized_corpus = []
for key in data:
    #alltexts += data[key][1]
    nowhitespacetext = re.sub('\s+',' ',data[key][1])
    alltexts.append(nowhitespacetext.replace(".", " ").replace(",", " ").replace(r" *", " ").replace("  ", " ").replace("– ",""))
    #print(data[key][1])
    #doc = nlp(data[key][1].replace('–', ''))
    #processedText = preprocessing(data[key][1].replace('–', ''))
    #print(preprocessing(data[key][1].replace('–', '')))
    #print(processedText)

In [73]:
print(alltexts[0])

sehr geehrter herr alterspräsident! so muss ich es jetzt sagen genau; ich muss mich daran gewöhnen nicht lange nicht lange; da haben sie schon recht liebe kolleginnen und kollegen! meine sehr geehrten damen und herren! die übernahme der geschäftsordnung ist eine der ersten wichtigen entscheidungen die wir heute zu beginn der wahlperiode treffen müssen die geschäftsordnung ist grundlage für unsere gemeinsame arbeit sie hat sich über viele wahlperioden bewährt und wird uns auch durch die neue wahlperiode tragen es ist gute tradition dass wir die geschäftsordnung am anfang einer wahlperiode mit breiter parlamentarischer mehrheit übernehmen und das sollten wir auch heute tun sie ist nicht in stein gemeißelt das wissen diejenigen die bereits in der letzten wahlperiode hier im bundestag tätig waren sondern die geschäftsordnung ist immer ein ich sage mal lebendiges dokument das sich den gegebenheiten des parlamentarismus aber auch den gegebenheiten der arbeit hier im deutschen bundestag anpas

In [77]:
for i, text in enumerate(alltexts):
    if text.find("me cfs") != -1:
        print("gefunden")
    if text.find("me/cfs") != -1:
        print("gefunden2")
    if text.find("me-cfs") != -1:
        print("gefunden3")
    if text.find(" me ") != -1:
        print("gefunden4", text)

gefunden2
gefunden2
gefunden2
gefunden4 sehr geehrte frau präsidentin! sehr geehrte damen und herren! lieber kollege stefinger ich bin erstaunt wie wenig sie diesem parlament zutrauen wir haben den haushalt noch gar nicht verabschiedet wir sitzen hier damit wir alle gemeinsam für die guten dinge und die guten ziele kämpfen und das werden wir im november entscheiden „let me learn!“ „lass mich lernen!“ das forderten weltweit jugendvertreter/‑innen und jugendbotschafter/-innen am wochenende stellvertretend für ihre generation beim bildungsgipfel der vereinten nationen in new york bildung ist ein menschenrecht in der realität ist sie jedoch ein privileg wir lassen viele kinder und jugendliche dieser welt nicht lernen; dieses in den sdgs und in der kinderrechtskonvention verankerte recht bleibt ihnen verwehrt bereits vor der coronapandemie hatten wir es mit einer bildungskrise zu tun; denn rund die hälfte der zehnjährigen kinder war nicht in der lage eine einfache geschichte zu lesen und si

In [79]:
extra_stops = ["tja","bereich", "wichtig", "herr", "frau", "kollegin", "kollege", "dame", "mal", "geehrte", "geehrt", "nächster", "redner", "lieb", "lieber", "liebe", "fraktionskollegin", "fraktionskollege", "lm", "sagen", "antrag", "letzter", "frage", "präsidentin", "wort"] 

In [95]:
# Laufzeit: ~1:45 Stunden
# 2. Laufzeit: 4 Stunden???
stop_words = set (stopwords.words("german"))
tokenized_corpus = []
for texts in alltexts:
    doc = nlp(texts)
    tokenized_texts = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num and token.lemma_.lower() not in stop_words and token.lemma_.lower() not in extra_stops and token.pos_ == "NOUN":
            tokenized_texts.append(token.lemma_.lower())
    
    tokenized_corpus.append(tokenized_texts)

In [115]:
#print(tokenized_corpus)

In [96]:
words_id = corpora.Dictionary(tokenized_corpus)
corpus = [words_id.doc2bow(txt) for txt in tokenized_corpus]

In [111]:
k_init = 10
k_final = 25

In [112]:
for k in range(k_init,k_final+1):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=words_id, num_topics = k, random_state=50, passes=40, per_word_topics=True)

    per_lda = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=words_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(k,per_lda,coherence_lda)

10 -9.023489059464659 0.49145697587328285
11 -9.220521102189455 0.5133971455145185
12 -9.419564411720163 0.49422386042861804
13 -9.586863627744888 0.5410719444473128
14 -9.7262448375729 0.5299777401062108
15 -9.849687334543313 0.5150894692902438
16 -9.962788641251775 0.543503398109923
17 -10.060591327919173 0.5261078631244478
18 -10.166932863055017 0.5601675461794763
19 -10.28035089478596 0.5221244363036371
20 -10.39339150370246 0.5333466953919006
21 -10.500746918691707 0.5128770267704991
22 -10.599713569412463 0.5433951936591307
23 -10.703950540932944 0.5490276796328728
24 -10.827547899650893 0.523066335326212
25 -10.923928540204747 0.5187488399672823


In [113]:
#Laufzeit: ~2 min
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=words_id, num_topics = 23, random_state=50, passes=40, per_word_topics=True)

In [114]:
lda_model.show_topics(num_words=20, num_topics=23)

[(0,
  '0.070*"kommune" + 0.055*"wohnung" + 0.025*"mieter" + 0.024*"wohnraum" + 0.019*"vorkaufsrecht" + 0.019*"stadt" + 0.018*"wohnen" + 0.017*"wohnungsbau" + 0.017*"mieterinnen" + 0.016*"städte" + 0.013*"bauen" + 0.011*"miete" + 0.011*"gemeinde" + 0.010*"sanierung" + 0.010*"eigentum" + 0.009*"ort" + 0.008*"wohnungsmarkt" + 0.007*"bauministerium" + 0.007*"sozialwohnung" + 0.007*"ministerin"'),
 (1,
  '0.010*"mindeststandard" + 0.008*"sympathie" + 0.004*"absurdität" + 0.004*"industrieunternehmen" + 0.003*"ernährungsarmut" + 0.003*"pate" + 0.003*"klimaaktivistinnen" + 0.003*"un-resolution" + 0.002*"genugtuung" + 0.002*"rundfunkbeitrag" + 0.002*"vorjahreszeitraum" + 0.002*"nutri-score" + 0.002*"besage" + 0.002*"‑soldaten" + 0.002*"politisierung" + 0.002*"rohstoffsicherung" + 0.002*"vorlagen" + 0.001*"faktum" + 0.001*"verfassungsrichter" + 0.001*"unterernährung"'),
 (2,
  '0.021*"mensch" + 0.019*"land" + 0.012*"bundesregierung" + 0.012*"krieg" + 0.011*"welt" + 0.010*"staat" + 0.010*"demokr

In [133]:
#lda_model.save('lda_model')
#lda_model.load()

from gensim.test.utils import datapath

temp_file = datapath("model_clara")
lda_model.save(temp_file)

In [131]:
lda_model[corpus[5500]]
# Corpus[2] im Datensatz checken und Textinhalt gegen Topics prüfen

([(3, 0.0122544505),
  (4, 0.109100014),
  (5, 0.029685762),
  (8, 0.07219328),
  (9, 0.0564028),
  (10, 0.438357),
  (13, 0.07657137),
  (15, 0.030211877),
  (17, 0.11809969),
  (20, 0.0508953)],
 [(7, [4, 13, 8, 20, 17, 9, 15]),
  (11, [4, 10, 13, 8, 15, 9, 17]),
  (17, [10, 17, 4, 20, 15]),
  (22, [10, 4, 17, 9, 20, 15]),
  (44, [4, 10, 9, 15, 17, 8]),
  (52, [4, 10, 9, 15, 13]),
  (60, [10, 17, 9, 20, 15, 8, 4, 13]),
  (75, [10, 4, 8, 9, 13, 15]),
  (179, [20, 10, 17, 13, 4, 8, 5, 15]),
  (192, [8, 10, 15]),
  (195, [4, 8]),
  (216, [13, 4, 10, 8, 9]),
  (229, [4, 17, 10, 20, 15]),
  (242, [10, 17, 20, 13, 15, 4, 9]),
  (262, [9, 15, 4, 10, 17, 20]),
  (278, [10, 9, 8, 20, 17, 4, 15]),
  (280, [10, 4, 17, 9, 13, 8, 5, 15]),
  (329, [10, 4, 20, 17, 15, 9]),
  (355, [10, 17, 8, 13, 4, 3, 15]),
  (373, [8]),
  (417, [10, 17, 4, 9, 8, 20, 15]),
  (487, [10, 13, 20, 15]),
  (501, [10, 17, 15, 9, 4]),
  (549, [8, 10, 17, 20, 15, 9]),
  (559, [10, 17, 8]),
  (607, [4, 17, 13, 8, 20, 10, 9

In [120]:
print(corpus[151])

[(3, 1), (5, 2), (11, 1), (60, 1), (72, 1), (117, 1), (136, 1), (185, 1), (210, 1), (229, 1), (252, 1), (285, 1), (323, 1), (334, 1), (417, 1), (437, 4), (542, 1), (549, 2), (559, 4), (621, 1), (642, 1), (661, 2), (662, 4), (674, 2), (717, 3), (738, 1), (790, 1), (802, 2), (820, 1), (913, 2), (939, 1), (1016, 1), (1084, 2), (1215, 1), (1462, 1), (1475, 1), (1476, 1), (1625, 1), (1744, 1), (1998, 1), (2214, 1), (2222, 5), (2591, 2), (2685, 1), (2732, 2), (2784, 1), (3247, 1), (3267, 1), (3790, 4), (3791, 1), (3792, 1), (3793, 1), (3794, 1), (3795, 1), (3796, 2), (3797, 1), (3798, 1), (3799, 1), (3800, 1), (3801, 1), (3802, 1), (3803, 1), (3804, 1), (3805, 1), (3806, 1), (3807, 1), (3808, 1)]


In [132]:
print(alltexts[5500])

 herr präsident! frau ministerin! werte kollegen! meine sehr verehrten damen und herren! das wichtige thema „die erheblichen lernrückstände sowie die psychosozialen folgen für unsere schüler und schülerinnen“ wird hier debattiert das sind wir unseren kindern der zukunft unseres landes auch dringend schuldig sehr schön liebe kollegen von der union dass die partei die für die entstandenen probleme mit regierungsverantwortlich zeichnet sich nach den ergebnissen ihrer politik erkundigt ich halte ihnen zugute werte kollegen dass sie dies mit dem ziel tun es künftig besser zu machen fassen wir kurz zusammen: unsere kinder wurden mit völlig überzogenen widersinnigen und schädlichen coronamaßnahmen gequält diese führten neben den physischen und psychischen schäden zu großen lernlücken besonders bei sozioökonomisch benachteiligten familien das neue infektionsschutzgesetz erlaubt dies erneut unter dem deckmäntelchen von corona meine damen und herren sie befinden sich auch hier auf einer ideologi